In [1]:
import os
os.chdir("../")

In [2]:
os.getcwd()

'c:\\Users\\Mohit\\Desktop\\Mohit\\Projects\\E2EDSProject1'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path


In [4]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories
from src.datascience.entity.config_entity import (DataIngestionconfig, DataValidationConfig)

class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionconfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionconfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config
    
    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema
        )

        return data_validation_config
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config
    
    


In [5]:
import os
from src.datascience import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [6]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    # Note: you can add differenr data transformation techniques such as Scaler, PCA etc..
    # you can perform all kinds of EDA in ML cycle here before passing this data to the model
    # I am only adding train_test_split because this data is already clean
    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path)

        # Split the data into training and test sets. (0.75L 0.25) split
        train, test = train_test_split(data, test_size=0.25, random_state=42)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Splitted data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [7]:
try: 
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()

except Exception as e:
    raise e

[2024-12-28 18:38:28,262: INFO: common: yaml file: config\config.yaml loaded sucessfully]
[2024-12-28 18:38:28,264: INFO: common: yaml file: params.yaml loaded sucessfully]
[2024-12-28 18:38:28,268: INFO: common: yaml file: schema.yaml loaded sucessfully]
[2024-12-28 18:38:28,269: INFO: common: Created directory at: artifacts]
[2024-12-28 18:38:28,284: INFO: common: Created directory at: artifacts/data_transformation]
[2024-12-28 18:38:28,313: INFO: 1108923817: Splitted data into training and test sets]
[2024-12-28 18:38:28,314: INFO: 1108923817: (1199, 12)]
[2024-12-28 18:38:28,316: INFO: 1108923817: (400, 12)]
(1199, 12)
(400, 12)
